**Importing the libraries**

In [21]:
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from eli5.sklearn import PermutationImportance
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor,Pool
from sklearn import preprocessing
import matplotlib.patches as patch
from scipy.stats import kurtosis
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import skew
from scipy.stats import norm
from scipy import linalg
from sklearn import tree
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
import graphviz
import warnings
import random
import eli5
from eli5.sklearn import PermutationImportance
import shap  # package used to calculate Shap values
import time
import glob
import sys
import os

**Python pre-requisites**

In [2]:
%matplotlib inline
%precision 4
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 15)

**Setting path for creating data pipeline**

In [3]:
path = 'D:\ISU\IE 587\Project\denoised features'
os.chdir(path)

**Loading the train data**

In [4]:
%%time
train = pd.read_csv('train_features_denoised.csv')

Wall time: 2.1 s


In [5]:
train.head(5)

,Hann_window_mean_150,Hann_window_mean_1500,Hann_window_mean_15000,Hann_window_mean_50,Hilbert_mean,abs_energy,abs_max,abs_max_roll_mean_10,abs_max_roll_mean_100,abs_max_roll_mean_1000,...,time_rev_asym_stat_1,time_rev_asym_stat_10,time_rev_asym_stat_100,time_rev_asym_stat_1000,time_rev_asym_stat_10000,time_rev_asym_stat_5,time_rev_asym_stat_50,time_rev_asym_stat_500,trend,var_larger_than_std_dev
0,-0.001664147605421,-0.001635852868712,-0.001575525520872,-0.001691640541782,1.571311178197287,2.214657520390557e+06,103.719286521655889,65.848238564092441,5.124388698175711,0.514386190173771,...,-0.096420407246901,-24.507658220827970,3.706442323830551,1.413534184174831,0.471447345824418,-6.521118219461393,-0.009140143692533,2.522993798144944,-0.000000060056403,True
1,-0.000537758286648,-0.000534781236927,-0.000506705333658,-0.000537978776103,2.056154069838052,4.763819837510224e+06,179.071606940658143,154.927312691455825,10.235463078671351,1.025659834311068,...,0.258199532798016,88.924023421070942,143.980043802316686,7.359462725006689,-2.108296571555000,84.043278984183502,68.351270883398328,6.802377071091023,-0.000000020051138,True
2,0.002995159477478,0.003000717476243,0.002878113542093,0.002969253456014,2.609251779225567,5.347760421955101e+06,132.997666959632937,92.656081225090844,6.994292010841567,0.671265332808759,...,-0.615191595512900,-24.766723941285051,65.447497488503188,0.486344666576969,6.692607147215793,-37.209306572942772,11.370507392062022,40.075131250635721,0.000000086074548,True
3,-0.000377213786939,-0.000366397326087,-0.000352430802049,-0.000381808657018,2.164196813553777,5.402907428912619e+06,212.923499302500488,134.717540078522433,14.527569077811634,1.003279179335431,...,1.888609274809342,214.061271017309423,235.203521470197302,-0.909736517939260,4.128007149771127,147.369930250765037,-82.615385161359043,21.555171209747233,-0.000000012875785,True
4,-0.002107701068876,-0.002085780327126,-0.001965103752775,-0.002108543206342,2.526496296389278,6.236539020102236e+06,145.860531645298181,99.522535237699316,8.743624387715270,0.848335352533279,...,-0.941679270691838,8.955586639683442,59.356235470642254,3.132654224243285,0.467576902184649,17.314863577612133,56.868139445440292,-31.466588602536980,-0.000000074737843,True


In [6]:
train.select_dtypes(include='object')

,seg_id
0,train_0
1,train_1
2,train_2
3,train_3
4,train_4
...,...
4190,train_4190
4191,train_4191
4192,train_4192
4193,train_4193


In [7]:
train = train.drop(['var_larger_than_std_dev','seg_id'], axis=1)

In [8]:
y = pd.read_csv('y.csv')

In [9]:
train.shape

(4195, 977)

**Loading the test data**

In [10]:
%%time
test = pd.read_csv('test_features_denoised.csv')

Wall time: 1.45 s


In [11]:
test = test.drop(['var_larger_than_std_dev','seg_id'], axis=1)

In [12]:
test.shape

(2624, 977)

**Creating submission file**

In [13]:
submission = pd.read_csv('sample_submission.csv', index_col='seg_id')
submission.head()

,time_to_failure
seg_id,
seg_00030f,0
seg_0012b5,0
seg_00184e,0
seg_003339,0
seg_0042cc,0


In [14]:
submission.shape

(2624, 1)

**Standardizing the train data**

In [15]:
scaler = StandardScaler()
scaler.fit(train)
X_scaled = scaler.transform(train)

In [16]:
X_scaled = pd.DataFrame(X_scaled)

**Standardizing the test data**

In [17]:
scaler = StandardScaler()
scaler.fit(test)
test_scaled = scaler.transform(test)

**Create data partition**

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, shuffle=True)

**Model training: SVM**

In [23]:
svm = NuSVR()
svm_fit = svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_val)

In [24]:
score = mean_absolute_error(y_val, y_pred_svm)
print(f'Score: {score:0.3f}')

Score: 1.267
